# Query Collatz Database

This notebook queries the PostgreSQL database to check if Collatz sequence data has been saved.

In [29]:
import os
import sys
import pandas as pd
from datetime import datetime

# Add src to path
sys.path.append(os.path.join('..', 'src'))

from collatz.database import CollatzDatabase

In [30]:
# Initialize database connection
db = CollatzDatabase()

# Test connection
try:
    with db:
        print('✅ Database connection successful!')
        
        # Check if tables exist
        cursor = db.connection.cursor()
        cursor.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public' 
            ORDER BY table_name;
        """)
        tables = cursor.fetchall()
        print(f'\nAvailable tables: {[table[0] for table in tables]}')
        cursor.close()
except Exception as e:
    print(f'❌ Database connection failed: {e}')

✅ Database connection successful!

Available tables: ['active_experiments', 'analysis_results', 'batch_job_progress', 'batch_jobs', 'collatz_sequences', 'daily_processing_stats', 'experiment_status_overview', 'experiments', 'performance_metrics', 'performance_summary', 'recent_analysis_summary', 'sequence_length_distribution', 'sequence_statistics', 'sequence_steps', 'top_longest_sequences', 'top_peak_sequences']


In [31]:
# Query collatz_sequences table
try:
    with db:
        cursor = db.connection.cursor()
        
        # Count total sequences
        cursor.execute('SELECT COUNT(*) FROM collatz_sequences;')
        total_count = cursor.fetchone()[0]
        print(f'📊 Total sequences in database: {total_count:,}')
        
        if total_count > 0:
            # Get range of starting numbers
            cursor.execute("""
                SELECT MIN(starting_number), MAX(starting_number) 
                FROM collatz_sequences;
            """)
            min_num, max_num = cursor.fetchone()
            print(f'📈 Range: {min_num} to {max_num}')
            
            # Get some sample data
            cursor.execute("""
                SELECT starting_number, sequence_length, max_value, 
                       total_steps, stopping_time, created_at
                FROM collatz_sequences 
                ORDER BY starting_number 
                LIMIT 10;
            """)
            
            results = cursor.fetchall()
            columns = ['starting_number', 'sequence_length', 'max_value', 
                      'total_steps', 'stopping_time', 'created_at']
            
            df_sample = pd.DataFrame(results, columns=columns)
            print('\n📋 Sample data (first 10 rows):')
            print(df_sample)
        
        cursor.close()
        
except Exception as e:
    print(f'❌ Error querying sequences: {e}')

📊 Total sequences in database: 10,001
📈 Range: 1 to 999999

📋 Sample data (first 10 rows):
   starting_number  sequence_length  max_value  total_steps  stopping_time  \
0                1                1          1            0              1   
1                2                2          2            0              2   
2                3                8         16            3              8   
3                4                3          4            0              3   
4                5                6         16            1              6   
5                6                9         16            4              9   
6                7               17         52            5             17   
7                8                4          8            0              4   
8                9               20         52            8             20   
9               10                7         16            2              7   

                        created_at  
0 2025-08-06 

In [32]:
# Query analysis_results table
try:
    with db:
        cursor = db.connection.cursor()
        
        # Count analysis results
        cursor.execute('SELECT COUNT(*) FROM analysis_results;')
        analysis_count = cursor.fetchone()[0]
        print(f'\n🔬 Total analysis results: {analysis_count}')
        
        if analysis_count > 0:
            # Get recent analysis results
            cursor.execute("""
                SELECT analysis_type, range_start, range_end, 
                       total_sequences, avg_sequence_length, 
                       max_sequence_length, created_at
                FROM analysis_results 
                ORDER BY created_at DESC 
                LIMIT 5;
            """)
            
            results = cursor.fetchall()
            columns = ['analysis_type', 'range_start', 'range_end', 
                      'total_sequences', 'avg_sequence_length', 
                      'max_sequence_length', 'created_at'] 
            
            df_analysis = pd.DataFrame(results, columns=columns)
            print('\n📊 Recent analysis results:')
            print(df_analysis)
        
        cursor.close()
        
except Exception as e:
    print(f'❌ Error querying analysis results: {e}')


🔬 Total analysis results: 3

📊 Recent analysis results:
           analysis_type  range_start  range_end  total_sequences  \
0    peak_value_analysis            1       1000             1000   
1  sequence_distribution            1        100              100   
2   basic_range_analysis            1         20               20   

  avg_sequence_length  max_sequence_length                       created_at  
0               15.23                  178 2025-08-06 23:40:45.755135+08:00  
1               12.45                  118 2025-08-06 23:40:45.755135+08:00  
2               10.15                   20 2025-08-06 23:40:45.755135+08:00  


In [33]:
# Get database statistics
try:
    with db:
        cursor = db.connection.cursor()
        
        # Get sequence statistics
        cursor.execute("""
            SELECT 
                AVG(sequence_length) as avg_length,
                MIN(sequence_length) as min_length,
                MAX(sequence_length) as max_length,
                AVG(max_value) as avg_max_value,
                MIN(max_value) as min_max_value,
                MAX(max_value) as max_max_value
            FROM collatz_sequences;
        """)
        
        stats = cursor.fetchone()
        if stats and stats[0] is not None:
            print('\n📈 Database Statistics:')
            print(f'Average sequence length: {stats[0]:.2f}')
            print(f'Sequence length range: {stats[1]} - {stats[2]}')
            print(f'Average max value: {stats[3]:.2f}')
            print(f'Max value range: {stats[4]} - {stats[5]}')
        
        cursor.close()
        
except Exception as e:
    print(f'❌ Error getting statistics: {e}')


📈 Database Statistics:
Average sequence length: 85.96
Sequence length range: 1 - 262
Average max value: 58990.59
Max value range: 1 - 27114424


In [34]:
# Check for recently added data
try:
    with db:
        cursor = db.connection.cursor()
        
        # Get data added in the last hour
        cursor.execute("""
            SELECT COUNT(*) 
            FROM collatz_sequences 
            WHERE created_at > NOW() - INTERVAL '1 hour';
        """)
        
        recent_count = cursor.fetchone()[0]
        print(f'\n⏰ Sequences added in the last hour: {recent_count}')
        
        # Get the most recent entries
        cursor.execute("""
            SELECT starting_number, sequence_length, created_at
            FROM collatz_sequences 
            ORDER BY created_at DESC 
            LIMIT 5;
        """)
        
        recent_data = cursor.fetchall()
        if recent_data:
            print('\n🕐 Most recent entries:')
            for row in recent_data:
                print(f'  Starting number {row[0]}: length {row[1]}, added at {row[2]}')
        
        cursor.close()
        
except Exception as e:
    print(f'❌ Error checking recent data: {e}')


⏰ Sequences added in the last hour: 9981

🕐 Most recent entries:
  Starting number 999999: length 10, added at 2025-08-07 03:25:24.706893+08:00
  Starting number 9979: length 74, added at 2025-08-07 02:45:13.884245+08:00
  Starting number 9978: length 136, added at 2025-08-07 02:45:13.884245+08:00
  Starting number 9961: length 74, added at 2025-08-07 02:45:13.884245+08:00
  Starting number 9960: length 92, added at 2025-08-07 02:45:13.884245+08:00
